In [6]:
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 根目录
root_path = '/home/ljy/kaggle/'

In [7]:
%matplotlib inline
import gc
import os
import itertools
import pickle
import re
import time
from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn import metrics, model_selection, preprocessing, linear_model, ensemble, decomposition, tree
import lightgbm as lgb
import torch
# import numpy as np
from scipy.stats import skew
import polars as pl
from sklearn.feature_extraction.text import TfidfVectorizer
from path import Path
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

In [95]:
# ==========增加新的特征==========
# Define the numerical and categorical variables as in Sub3
num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']

# Reuse the count_by_values function from Sub3 as it is
def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts = fts.join(tmp_df, on='id', how='left') 
    return fts

def adjusted_dev_feats(df):
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    print("< Input words stats features >")
    temp = df.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
    temp = temp.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
    temp = temp.with_columns(input_word_count = pl.col('text_change').list.lengths(),
                             input_word_length_mean = pl.col('text_change').apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max = pl.col('text_change').apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std = pl.col('text_change').apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median = pl.col('text_change').apply(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew = pl.col('text_change').apply(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
    temp = temp.drop('text_change')
    feats = feats.join(temp, on='id', how='left') 

    print("< Numerical columns features >")
    temp = df.group_by("id").agg(pl.sum('action_time').suffix('_sum'), pl.mean(num_cols).suffix('_mean'), pl.std(num_cols).suffix('_std'),
                                 pl.median(num_cols).suffix('_median'), pl.min(num_cols).suffix('_min'), pl.max(num_cols).suffix('_max'),
                                 pl.quantile(num_cols, 0.5).suffix('_quantile'))
    feats = feats.join(temp, on='id', how='left') 

    print("< Categorical columns features >")
    temp  = df.group_by("id").agg(pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
    feats = feats.join(temp, on='id', how='left') 

    print("< Idle time features >")
    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.group_by("id").agg(inter_key_largest_latency = pl.max('time_diff'),
                                   inter_key_median_latency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter(pl.col('time_diff') > 3).count())
    feats = feats.join(temp, on='id', how='left') 

    print("< P-bursts features >")
    #找到df中activity为‘Input’和‘Remove/cut’的行,并且是time_diff<2的行
    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('time_diff')<2)
    #然后统计连续出现的个数的(统计学变量)
    temp = temp.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
    temp = temp.drop_nulls()#删除包含缺失值的行
    temp = temp.group_by("id").agg(pl.mean('P-bursts').suffix('_mean'), pl.std('P-bursts').suffix('_std'), pl.count('P-bursts').suffix('_count'),
                                   pl.median('P-bursts').suffix('_median'), pl.max('P-bursts').suffix('_max'),
                                   pl.first('P-bursts').suffix('_first'), pl.last('P-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left') 

    print("< R-bursts features >")
    #取出数据中为'Remove/cut'
    temp = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('activity').is_in(['Remove/Cut']))
    #统计'Remove/cut'连续出现的次数(的统计学变量)
    temp = temp.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
    temp = temp.drop_nulls()#删除包含缺失值的行
    temp = temp.group_by("id").agg(pl.mean('R-bursts').suffix('_mean'), pl.std('R-bursts').suffix('_std'), 
                                   pl.median('R-bursts').suffix('_median'), pl.max('R-bursts').suffix('_max'),
                                   pl.first('R-bursts').suffix('_first'), pl.last('R-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left')
    
    return feats

train_logs    = pl.scan_csv(root_path + '/input/linking-writing-processes-to-writing-quality/train_logs.csv')

test_logs=pd.read_csv(root_path + "/input/linking-writing-processes-to-writing-quality/test_logs.csv")
print(f"len(test_logs):{len(test_logs)}")
test_logs=test_logs.sort_values(by=['id', 'down_time'])
# 重置索引
test_logs = test_logs.reset_index(drop=True)
# 根据'id'列进行分组，并为每个分组添加一个递增的序列
test_logs['event_id'] = test_logs.groupby('id').cumcount() + 1
test_logs.to_csv("test_logs.csv",index=None)
test_logs   = pl.scan_csv(root_path + '/working/test_logs.csv')

# Assume burst features are calculated using the adjusted dev_feats function.
burst_feats_train = adjusted_dev_feats(train_logs)  # Adjusted function with train data
burst_feats_train = burst_feats_train.collect().to_pandas()
burst_feats_test = adjusted_dev_feats(test_logs)    # Adjusted function with test data
burst_feats_test = burst_feats_test.collect().to_pandas()

# Function to construct essays copied from here (small adjustments): https://www.kaggle.com/code/kawaiicoderuwu/essay-contructor

def getEssays(df):
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']]
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    lastIndex = 0
    essaySeries = pd.Series()
    for index, valCount in enumerate(valCountsArr):
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]
        lastIndex += valCount
        essayText = ""
        for Input in currTextInput.values:
            if Input[0] == 'Replace':
                replaceTxt = Input[2].split(' => ')
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] +\
                essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue
            if Input[0] == 'Paste':
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue
            if Input[0] == 'Remove/Cut':
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue
            if "M" in Input[0]:
                croppedTxt = Input[0][10:]
                splitTxt = croppedTxt.split(' To ')
                valueArr = [item.split(', ') for item in splitTxt]
                moveData = (int(valueArr[0][0][1:]), 
                            int(valueArr[0][1][:-1]), 
                            int(valueArr[1][0][1:]), 
                            int(valueArr[1][1][:-1]))
                if moveData[0] != moveData[2]:
                    if moveData[0] < moveData[2]:
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] +\
                        essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] +\
                        essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        essaySeries[index] = essayText
    essaySeries.index =  textInputDf['id'].unique()
#     return pd.DataFrame(essaySeries, columns=['essay'])
    return pd.DataFrame(essaySeries, columns=['essay']).reset_index().rename(columns={"index":'id'})

# Helper functions
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

AGGREGATIONS = ['count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']

# v5: add word_aggregations to sub1 train_feats before splitting into kinds of train_feats
#将论文转成单词
def split_essays_into_words(df):
    essay_df = df
    #对空格,\n,句号问号感叹号进行匹配,得到一个拆分后的列表.
    essay_df['word'] = essay_df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    # essay1 [1,2,3] essay2[4,5] ->5行 essay1 1  // essay1 2 // essay1 3 // essay2 1 // essay2 2
    essay_df = essay_df.explode('word')
    #求出每个单词的长度
    essay_df['word_len'] = essay_df['word'].apply(lambda x: len(x))
    #去掉单词长度为0的数据
    essay_df = essay_df[essay_df['word_len'] != 0]
    return essay_df

#计算word_len的统计学变量,并计算>=word_len的词数
def compute_word_aggregations(word_df):
    #根据id计算单词长度的统计学变量
    word_agg_df = word_df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    #比如('mean','word_len')->'mean_word_len'
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    for word_l in [5, 6, 7, 8, 9, 10, 11, 12]:
        #ge 就是Latex里>=的符号,筛选出word_len>=word_l的行,根据id进行统计,提取每个计数的第0行
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_df[word_df['word_len'] >= word_l].groupby(['id']).count().iloc[:, 0]
        #如果有缺失值就填充为0
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_agg_df[f'word_len_ge_{word_l}_count'].fillna(0)
    #重置索引
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df

# v5: Word features for train dataset
train_essays = pd.read_csv(root_path + '/input/writing-quality-challenge-constructed-essays/train_essays_fast.csv')
train_word_agg_df = compute_word_aggregations(split_essays_into_words(train_essays))

test_logs   = pl.scan_csv(root_path + '/working/test_logs.csv')
test_logs             = test_logs.collect().to_pandas()

test_essays = getEssays(test_logs)
test_essays_copy=test_essays.copy()

# v5: Word features for test dataset
test_word_agg_df = compute_word_aggregations(split_essays_into_words(test_essays))

# v6: add two new features from sub3 to sub1 feats_lgbm:
# 1. Ratio of final product length to keys pressed.
# 2. Keys per second.

#论文的长度除以(‘Input’和‘Remove/Cut’)的按键个数.
def product_to_keys(logs, essays):
    essays['product_len'] = essays.essay.str.len()#论文的长度
    #logs中每个id ‘Input’和‘Remove/Cut’的数据
    tmp_df = logs[logs.activity.isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
    essays = essays.merge(tmp_df, on='id', how='left')
    #论文的长度除以(‘Input’和‘Remove/Cut’)的按键个数.
    essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
    return essays[['id', 'product_to_keys']]

#统计每秒有几个['Input', 'Remove/Cut']的行为.
def get_keys_pressed_per_second(logs):
    #logs中为['Input', 'Remove/Cut']的event_id的个数
    temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
    #每个id最小的down_time和最大的up_time
    temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
    #按照id融合在一起
    temp_df = temp_df.merge(temp_df_2, on='id', how='left')
    #每秒有几个event_id
    temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
    return temp_df[['id', 'keys_per_second']]

# v6: 
train_logs    = pl.scan_csv(root_path + '/input/linking-writing-processes-to-writing-quality/train_logs.csv')
train_logs             = train_logs.collect().to_pandas()
train_essays           = pd.read_csv(root_path + '/input/writing-quality-challenge-constructed-essays/train_essays_fast.csv')
test_logs   = pl.scan_csv(root_path + '/working/test_logs.csv')
test_logs             = test_logs.collect().to_pandas()
test_essays           = test_essays_copy
keys_pressed_per_second_feats_train = get_keys_pressed_per_second(train_logs)
product_to_keys_feats_train = product_to_keys(train_logs, train_essays)

keys_pressed_per_second_feats_test = get_keys_pressed_per_second(test_logs)
product_to_keys_feats_test = product_to_keys(test_logs, test_essays)

INPUT_DIR = '../input/linking-writing-processes-to-writing-quality'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

# lgbm-x2-nn-fusion-newversion-baseline1-v1
def Get_Event_Counts(df):
    Nonproduction_num = df['activity'].value_counts().get('Nonproduction', 0)
    Input_num = df['activity'].value_counts().get('Input', 0)
    Replace_num = df['activity'].value_counts().get('Replace', 0)
    Remove_orCut_num = df['activity'].value_counts().get('Remove/Cut', 0)
    Paste_num = df['activity'].value_counts().get('Paste', 0)
    # 不在前五类的定义为 'Move From'
    Move_From_num = df['activity'].value_counts().loc[lambda x: ~x.index.isin(['Nonproduction', 'Input', 'Replace', 'Remove/Cut', 'Paste'])].sum()

    return Nonproduction_num,Input_num,Replace_num,Remove_orCut_num,Paste_num, Move_From_num

def Get_Activity_Sum(df):
    return df.shape[0]

def Get_Activity_Essay(df):
    filtered_data = df['activity']
    text = ' '.join(filtered_data.astype(str))
    return text

def Latency_Features(df):
    df['activity'] = df['activity'].replace(to_replace=r'^Move From.*$', value='Move From', regex=True)

    # 使用 groupby 得到每种活动类型的最小值、最大值和平均值
    min_action_time = df.groupby('activity')['action_time'].min()
    max_action_time = df.groupby('activity')['action_time'].max()
    mean_action_time = df.groupby('activity')['action_time'].mean()

    # 使用 get 方法获取各个活动类型的值，如果不存在则返回默认值（比如 NaN）
    min_input_time = min_action_time.get('Input', None)
    min_move_time = min_action_time.get('Move From', None)
    min_Nonproduction_time = min_action_time.get('Nonproduction', None)
    min_Paste_time = min_action_time.get('Paste', None)
    min_RemoveOrCut_time = min_action_time.get('Remove/Cut', None)

    max_input_time = max_action_time.get('Input', None)
    max_move_time = max_action_time.get('Move From', None)
    max_Nonproduction_time = max_action_time.get('Nonproduction', None)
    max_Paste_time = max_action_time.get('Paste', None)
    max_RemoveOrCut_time = max_action_time.get('Remove/Cut', None)

    mean_input_time = mean_action_time.get('Input', None)
    mean_move_time = mean_action_time.get('Move From', None)
    mean_Nonproduction_time = mean_action_time.get('Nonproduction', None)
    mean_Paste_time = mean_action_time.get('Paste', None)
    mean_RemoveOrCut_time = mean_action_time.get('Remove/Cut', None)

    return min_input_time, min_move_time, min_Nonproduction_time, min_Paste_time, min_RemoveOrCut_time, \
           max_input_time, max_move_time, max_Nonproduction_time, max_Paste_time, max_RemoveOrCut_time, \
           mean_input_time, mean_move_time, mean_Nonproduction_time, mean_Paste_time, mean_RemoveOrCut_time



def Generate_data(df):
    warnings.simplefilter('ignore')
    
    grouped = df.groupby('id')

    # 创建一个字典，用于存储每个组的数据
    grouped_data = {}

    # 获取所有分组的信息
    group_info = grouped.groups

    # 遍历每个分组，存储数据到字典
    for group_label, group_index in group_info.items():
        group_data = df.loc[group_index]
        group_name = f"{group_label}"  # 根据分组的名称进行命名
        grouped_data[group_name] = group_data
        
    data = pd.DataFrame()

    for group_name, group_data in tqdm(grouped_data.items()):
        id = group_name
        Activity_Essay = Get_Activity_Essay(group_data)
        Activity_Count = Get_Activity_Sum(group_data)
        Nonproduction_num, Input_num, Replace_num, Remove_orCut_num, Paste_num, Move_From_num = Get_Event_Counts(group_data)
        min_input_time, min_move_time, min_Nonproduction_time, min_Paste_time, min_RemoveOrCut_time, \
               max_input_time, max_move_time, max_Nonproduction_time, max_Paste_time, max_RemoveOrCut_time, \
               mean_input_time, mean_move_time, mean_Nonproduction_time, mean_Paste_time, mean_RemoveOrCut_time = Latency_Features(group_data)
        # Append data to DataFrame
        # Append data to DataFrame
        # 向 DataFrame 添加数据
        new_row = {
            'id': id,
            'Activity_Essay': Activity_Essay or "",  # 如果 Activity_Essay 是 None，则提供一个空字符串
            'Activity_Count': Activity_Count or 0,   # 如果 Activity_Count 是 None，则提供 0
            'Nonproduction_num': Nonproduction_num or 0,
            'Input_num': Input_num or 0,
            'Replace_num': Replace_num or 0,
            'Remove_orCut_num': Remove_orCut_num or 0,
            'Paste_num': Paste_num or 0,
            'Move_From_num': Move_From_num or 0,
            'min_input_time': float(min_input_time) if min_input_time is not None else 0.0,  # 如果 min_input_time 是 None，则提供 0.0
            'min_move_time': float(min_move_time) if min_move_time is not None else 0.0,
            'min_Nonproduction_time': float(min_Nonproduction_time) if min_Nonproduction_time is not None else 0.0,
            'min_Paste_time': float(min_Paste_time) if min_Paste_time is not None else 0.0,
            'min_RemoveOrCut_time': float(min_RemoveOrCut_time) if min_RemoveOrCut_time is not None else 0.0,
            'max_input_time': float(max_input_time) if max_input_time is not None else 0.0,
            'max_move_time': float(max_move_time) if max_move_time is not None else 0.0,
            'max_Nonproduction_time': float(max_Nonproduction_time) if max_Nonproduction_time is not None else 0.0,
            'max_Paste_time': float(max_Paste_time) if max_Paste_time is not None else 0.0,
            'max_RemoveOrCut_time': float(max_RemoveOrCut_time) if max_RemoveOrCut_time is not None else 0.0,
            'mean_input_time': float(mean_input_time) if mean_input_time is not None else 0.0,
            'mean_move_time': float(mean_move_time) if mean_move_time is not None else 0.0,
            'mean_Nonproduction_time': float(mean_Nonproduction_time) if mean_Nonproduction_time is not None else 0.0,
            'mean_Paste_time': float(mean_Paste_time) if mean_Paste_time is not None else 0.0,
            'mean_RemoveOrCut_time': float(mean_RemoveOrCut_time) if mean_RemoveOrCut_time is not None else 0.0
        }


        data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
    
#     data = extract_skip_character_FromEssay(data)
#     data = Get_X_words(data)
    
    return data

# lgbm-x2-nn-fusion-newversion-baseline1-v2
# Activity_Essay, min_move_time, min_Paste_time, min_RemoveOrCut_time, max_move_time, max_Paste_time, max_RemoveOrCut_time, essay



def Generate_TFIDF_Features(data, test_data):
    
    tfidf = TfidfVectorizer()
    TFIDF_Activity_Essay = tfidf.fit_transform(data['Activity_Essay'])
    TFIDF_Activity_Essay_test = tfidf.transform(test_data['Activity_Essay'])
    
    tfidf = TfidfVectorizer()
    TFIDF_essay = tfidf.fit_transform(data['essay'])
    TFIDF_essay_test = tfidf.transform(test_data['essay'])

    data_combined = pd.concat([
        pd.DataFrame(TFIDF_Activity_Essay.toarray(), columns=[f'tfidf_Activity_{i}' for i in range(TFIDF_Activity_Essay.shape[1])]),
        pd.DataFrame(TFIDF_essay.toarray(), columns=[f'tfidf_essay_{i}' for i in range(TFIDF_essay.shape[1])]),
        data
    ], axis=1)
    
    test_data_combined = pd.concat([
        pd.DataFrame(TFIDF_Activity_Essay_test.toarray(), columns=[f'tfidf_Activity_{i}' for i in range(TFIDF_Activity_Essay.shape[1])]),
        pd.DataFrame(TFIDF_essay_test.toarray(), columns=[f'tfidf_essay_{i}' for i in range(TFIDF_essay.shape[1])]),
        test_data
    ], axis=1)
    
#     data_combined = data_combined.drop(columns=['Activity_Essay','essay','Unnamed: 0'])
#     test_data_combined = test_data_combined.drop(columns=['Activity_Essay','essay','Unnamed: 0'])

    return data_combined,test_data_combined

def Replace_NaN(data):
    data['min_move_time'] = data['min_move_time'].fillna(0)
    data['min_Paste_time'] = data['min_Paste_time'].fillna(0)
    data['min_RemoveOrCut_time'] = data['min_RemoveOrCut_time'].fillna(0)
    data['max_move_time'] = data['max_move_time'].fillna(0)
    data['max_Paste_time'] = data['max_Paste_time'].fillna(0)
    data['max_RemoveOrCut_time'] = data['max_RemoveOrCut_time'].fillna(0)
    return data

## Generate our features
test_data = Generate_data(test_logs)
train_data = Generate_data(train_logs)

train_data = train_data.merge(train_essays, on='id', how='left')
test_data = test_data.merge(test_essays, on='id', how='left')
train_data, test_data =  Generate_TFIDF_Features(train_data,test_data)

train_data.drop(columns=["essay"], inplace=True)
test_data.drop(columns=["essay"], inplace=True)

train_data =  Replace_NaN(train_data)
test_data =  Replace_NaN(test_data)

len(test_logs):6
< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >
< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >


100%|██████████| 2471/2471 [00:21<00:00, 113.16it/s]


In [96]:


# 更改数据路径
INPUT_DIR = '../input/linking-writing-processes-to-writing-quality'
INPUT_DIR2 = Path(root_path + '/input/writing-quality-challenge-constructed-essays')

# 模型保存路径
MODEL_DIR = Path(root_path + '/model/')

# 特征重要度门槛 or 特征数量门槛
fea_score_door = 0

In [97]:
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

In [98]:
train_essays = pd.read_csv(f'{INPUT_DIR2}/train_essays_02.csv')
train_essays.index = train_essays["Unnamed: 0"]
train_essays.index.name = None
train_essays.drop(columns=["Unnamed: 0"], inplace=True)
train_essays.head()

essay
001519c8  qqqqqqqqq qq qqqqq qq qqqq qqqq.  qqqqqq qqq q...
0022f953  qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq...
0042269b  qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
0059420b  qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
0075873a  qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq...

In [99]:
# 修改了这个函数 使得不会在部分版本上报错
# def getEssays(df):
#     textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']]
#     textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']
#     valCountsArr = textInputDf['id'].value_counts(sort=False).values
#     lastIndex = 0
#     essayList = []
#     for index, valCount in enumerate(valCountsArr):
#         currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]
#         lastIndex += valCount
#         essayText = ""
#         for Input in currTextInput.values:
#             if Input[0] == 'Replace':
#                 replaceTxt = Input[2].split(' => ')
#                 essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] +\
#                 essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
#                 continue
#             if Input[0] == 'Paste':
#                 essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
#                 continue
#             if Input[0] == 'Remove/Cut':
#                 essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
#                 continue
#             if "M" in Input[0]:
#                 croppedTxt = Input[0][10:]
#                 splitTxt = croppedTxt.split(' To ')
#                 valueArr = [item.split(', ') for item in splitTxt]
#                 moveData = (int(valueArr[0][0][1:]), 
#                             int(valueArr[0][1][:-1]), 
#                             int(valueArr[1][0][1:]), 
#                             int(valueArr[1][1][:-1]))
#                 if moveData[0] != moveData[2]:
#                     if moveData[0] < moveData[2]:
#                         essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] +\
#                         essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
#                     else:
#                         essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] +\
#                         essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
#                 continue
#             essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
#         essayList.append(essayText)
#     essaySeries = pd.Series(essayList)
#     essaySeries.index =  textInputDf['id'].unique()
#     return pd.DataFrame(essaySeries, columns=['essay'])

In [100]:
# Function to construct essays copied from here (small adjustments): https://www.kaggle.com/code/kawaiicoderuwu/essay-contructor

def getEssays(df):
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']]
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    lastIndex = 0
    essaySeries = pd.Series()
    for index, valCount in enumerate(valCountsArr):
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]
        lastIndex += valCount
        essayText = ""
        for Input in currTextInput.values:
            if Input[0] == 'Replace':
                replaceTxt = Input[2].split(' => ')
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] +\
                essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue
            if Input[0] == 'Paste':
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue
            if Input[0] == 'Remove/Cut':
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue
            if "M" in Input[0]:
                croppedTxt = Input[0][10:]
                splitTxt = croppedTxt.split(' To ')
                valueArr = [item.split(', ') for item in splitTxt]
                moveData = (int(valueArr[0][0][1:]), 
                            int(valueArr[0][1][:-1]), 
                            int(valueArr[1][0][1:]), 
                            int(valueArr[1][1][:-1]))
                if moveData[0] != moveData[2]:
                    if moveData[0] < moveData[2]:
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] +\
                        essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] +\
                        essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        essaySeries[index] = essayText
    essaySeries.index =  textInputDf['id'].unique()
    return pd.DataFrame(essaySeries, columns=['essay'])
#     return pd.DataFrame(essaySeries, columns=['essay']).reset_index().rename(columns={"index":'id'})

In [101]:
# Helper functions

def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [102]:
AGGREGATIONS = ['count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']

def split_essays_into_sentences(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    return essay_df

def compute_sentence_aggregations(df):
    sent_agg_df = pd.concat(
        [df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    )
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df

def split_essays_into_paragraphs(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

def compute_paragraph_aggregations(df):
    paragraph_agg_df = pd.concat(
        [df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    ) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

In [103]:
# Sentence features for train dataset
train_sent_df = split_essays_into_sentences(train_essays)
train_sent_agg_df = compute_sentence_aggregations(train_sent_df)

# Paragraph features for train dataset
train_paragraph_df = split_essays_into_paragraphs(train_essays)
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

In [104]:
# Features for test dataset
test_essays = getEssays(test_logs)
test_sent_agg_df = compute_sentence_aggregations(split_essays_into_sentences(test_essays))
test_paragraph_agg_df = compute_paragraph_aggregations(split_essays_into_paragraphs(test_essays))

In [105]:
# The following code comes almost Abdullah's notebook: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs
# Abdullah's code is based on work shared in previous notebooks (e.g., https://www.kaggle.com/code/hengzheng/link-writing-simple-lgbm-baseline)

from collections import defaultdict

class Preprocessor:
    
    def __init__(self, seed):
        self.seed = seed
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]
        
        self.idf = defaultdict(float)
    
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret

    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret

    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf
            
        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret

    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def make_feats(self, df):
        
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        print("Engineering time data")
        for gap in self.gaps:
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)

        print("Engineering cursor position data")
        for gap in self.gaps:
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        print("Engineering word count data")
        for gap in self.gaps:
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        print("Engineering statistical summaries for features")
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'quantile', 'sem', 'mean']),
            ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean'])]
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt])
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')

        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['up_time_max']

        return feats

In [106]:
preprocessor = Preprocessor(seed=42)
train_feats = preprocessor.make_feats(train_logs)
test_feats = preprocessor.make_feats(test_logs)
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()
train_feats = train_feats.drop(columns=nan_cols)
test_feats = test_feats.drop(columns=nan_cols)

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [02:52<00:00,  5.24s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 6451.81it/s]


Engineering event counts data


...

Engineering text change counts data


100%|██████████| 3/3 [00:00<00:00, 22192.08it/s]


Engineering punctuation counts data


100%|██████████| 3/3 [00:00<00:00, 33734.35it/s]


Engineering input words data
Engineering ratios data


In [107]:
# Code for additional aggregations comes from here: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs

train_agg_fe_df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
    ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
train_agg_fe_df.reset_index(inplace=True)

test_agg_fe_df = test_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
    ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

train_feats = train_feats.merge(train_agg_fe_df, on='id', how='left')
test_feats = test_feats.merge(test_agg_fe_df, on='id', how='left')

In [108]:
# Code for creating these features comes from here: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs
# Idea is based on features introduced in Section 3 of this research paper: https://files.eric.ed.gov/fulltext/ED592674.pdf

data = []

for logs in [train_logs, test_logs]:
    logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
    logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000

    group = logs.groupby('id')['time_diff']
    largest_lantency = group.max()
    smallest_lantency = group.min()
    median_lantency = group.median()
    initial_pause = logs.groupby('id')['down_time'].first() / 1000
    pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
    pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
    pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
    pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
    pauses_3_sec = group.apply(lambda x: (x > 3).sum())

    data.append(pd.DataFrame({
        'id': logs['id'].unique(),
        'largest_lantency': largest_lantency,
        'smallest_lantency': smallest_lantency,
        'median_lantency': median_lantency,
        'initial_pause': initial_pause,
        'pauses_half_sec': pauses_half_sec,
        'pauses_1_sec': pauses_1_sec,
        'pauses_1_half_sec': pauses_1_half_sec,
        'pauses_2_sec': pauses_2_sec,
        'pauses_3_sec': pauses_3_sec,
    }).reset_index(drop=True))

train_eD592674, test_eD592674 = data

train_feats = train_feats.merge(train_eD592674, on='id', how='left')
test_feats = test_feats.merge(test_eD592674, on='id', how='left')
train_feats = train_feats.merge(train_scores, on='id', how='left')

In [109]:
# Adding the additional features to the original feature set

train_feats = train_feats.merge(train_sent_agg_df, on='id', how='left')
train_feats = train_feats.merge(train_paragraph_agg_df, on='id', how='left')
test_feats = test_feats.merge(test_sent_agg_df, on='id', how='left')
test_feats = test_feats.merge(test_paragraph_agg_df, on='id', how='left')

In [110]:
#找到只有唯一值的列,删掉
keys=train_feats.keys().values
unique_cols=[key for key in keys if train_feats[key].nunique()<2]
print(f"unique_cols:{unique_cols}")
train_feats = train_feats.drop(columns=unique_cols)
test_feats = test_feats.drop(columns=unique_cols)

unique_cols:['word_count_change1_quantile', 'word_count_change2_quantile', 'tmp_cursor_position_min', 'smallest_lantency']


In [200]:
target_col = ['score']
drop_cols = ['id']
train_cols = [col for col in train_feats.columns if col not in target_col + drop_cols]

In [112]:
# # v1
# VER = 'v1'
# 
# Feature_DIR = Path('/home/ljy/kaggle/Feature/' + VER)
# if not Feature_DIR.exists():
#     os.mkdir(Feature_DIR)
# 
# MODEL_DIR = Path('/home/ljy/kaggle/model/' + VER)
# if not MODEL_DIR.exists():
#     os.mkdir(MODEL_DIR)
# 
# train_feats.to_csv(Feature_DIR / f'train_feats_{VER}.csv', index=False)
# test_feats.to_csv(Feature_DIR / f'test_feats_{VER}.csv', index=False)
# # train_feats = pd.read_csv(Feature_DIR / f'train_feats_{VER}.csv')
# # test_feats = pd.read_csv(Feature_DIR / f'test_feats_{VER}.csv')
# 
# train_feats.shape

In [113]:
# # v2
# VER = 'v2'
# 
# Feature_DIR = Path('/home/ljy/kaggle/Feature/' + VER)
# if not Feature_DIR.exists():
#     os.mkdir(Feature_DIR)
# 
# MODEL_DIR = Path('/home/ljy/kaggle/model/' + VER)
# if not MODEL_DIR.exists():
#     os.mkdir(MODEL_DIR)
# 
# # 特征重要度门槛 or 特征数量门槛
# fea_score_door = 10
# # fea_nums_door = 100
# 
# # 把上一次训练的部分重要度较低的特征去掉，再存入新的路径
# train_feats = train_feats[feature_importance_df.query('importance > @fea_score_door')['name']]
# test_feats = test_feats[feature_importance_df.query('importance > @fea_score_door')['name']]
# train_feats.to_csv(Feature_DIR / f'train_feats_{VER}.csv', index=False)
# test_feats.to_csv(Feature_DIR / f'test_feats_{VER}.csv', index=False)
# 
# test_feats.shape

In [131]:
# # v3，增加新的feature进来
# VER = 'v3'
# 
# # Merge burst features (adjusted_dev_feats)
# train_feats = burst_feats_train.merge(train_feats, on='id', how='left')
# test_feats = burst_feats_test.merge(test_feats, on='id', how='left')
# 
# # v5: word aggregation
# train_feats = train_word_agg_df.merge(train_feats, on='id', how='left')
# test_feats = test_word_agg_df.merge(test_feats, on='id', how='left')
# 
# # v6: keys_pressed_per_second and product_to_keys
# train_feats = keys_pressed_per_second_feats_train.merge(train_feats, on='id', how='left')
# train_feats = product_to_keys_feats_train.merge(train_feats, on='id', how='left')
# test_feats = keys_pressed_per_second_feats_test.merge(test_feats, on='id', how='left')
# test_feats = product_to_keys_feats_test.merge(test_feats, on='id', how='left')
# 
# # lgbm-x2-nn-fusion-newversion-baseline1: Out own new features
# train_feats = train_data.merge(train_feats, on='id', how='left')
# test_feats = test_data.merge(test_feats, on='id', how='left')
# 
# # ValueError: DataFrame.dtypes for data must be int, float or bool.
# # Did not expect the data types in the following fields: Activity_Essay
# train_feats = train_feats.drop(columns='Activity_Essay')
# test_feats = test_feats.drop(columns='Activity_Essay')
# 
# #找到只有唯一值的列,删掉
# keys=train_feats.keys().values
# unique_cols=[key for key in keys if train_feats[key].nunique()<2]
# print(f"unique_cols:{unique_cols}")
# train_feats = train_feats.drop(columns=unique_cols)
# test_feats = test_feats.drop(columns=unique_cols)
# 
# Feature_DIR = Path('/home/ljy/kaggle/Feature/' + VER)
# if not Feature_DIR.exists():
#     os.mkdir(Feature_DIR)
# 
# MODEL_DIR = Path('/home/ljy/kaggle/model/' + VER)
# if not MODEL_DIR.exists():
#     os.mkdir(MODEL_DIR)
# 
# # train_feats.to_csv(Feature_DIR / f'train_feats_{VER}.csv', index=False)
# # test_feats.to_csv(Feature_DIR / f'test_feats_{VER}.csv', index=False)
# train_feats = pd.read_csv(Feature_DIR / f'train_feats_{VER}.csv')
# test_feats = pd.read_csv(Feature_DIR / f'test_feats_{VER}.csv')
# 
# train_feats.shape
# test_feats.shape

unique_cols:['cursor_position_min']


(2471, 735)

(3, 734)

In [265]:
# v4

VER = 'v3'

Feature_DIR = Path('/home/ljy/kaggle/Feature/' + VER)
if not Feature_DIR.exists():
    os.mkdir(Feature_DIR)

MODEL_DIR = Path('/home/ljy/kaggle/model/' + VER)
if not MODEL_DIR.exists():
    os.mkdir(MODEL_DIR)

# 特征重要度门槛 or 特征数量门槛
# fea_score_door = 10
# fea_nums_door = 100

# # 把上一次训练的部分重要度较低的特征去掉，再存入新的路径
# train_feats = train_feats[feature_importance_df.query('importance > @fea_score_door')['name']]
# test_feats = test_feats[feature_importance_df.query('importance > @fea_score_door')['name']]
# train_feats.to_csv(Feature_DIR / f'train_feats_{VER}.csv', index=False)
# test_feats.to_csv(Feature_DIR / f'test_feats_{VER}.csv', index=False)

train_feats = pd.read_csv(Feature_DIR / f'train_feats_{VER}.csv')
test_feats = pd.read_csv(Feature_DIR / f'test_feats_{VER}.csv')

train_feats = train_feats.replace([np.inf, -np.inf], 1).dropna(axis=1, how='any')
train_feats = train_feats.drop(train_feats.filter(like='tfidf').columns, axis=1)

test_feats = test_feats.replace([np.inf, -np.inf], 1).dropna(axis=1, how='any')
test_feats = test_feats.drop(test_feats.filter(like='tfidf').columns, axis=1)

# train_feats = pd.concat([train_feats.iloc[:, 2:5][0:5], train_feats["score"]], axis=1)
# test_feats = test_feats.iloc[:, 2:5][0:5]
# train_feats = train_feats.dropna()

train_feats.shape, test_feats.shape

((2471, 531), (3, 26))

In [266]:
train_cols = [col for col in train_feats.columns if col not in target_col + drop_cols]

In [272]:
# 模型训练
import optuna
from optuna.samplers import TPESampler
from sklearn import model_selection, metrics
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import joblib

OOF_PREDS = np.zeros((len(train_feats), 2))
TEST_PREDS = np.zeros((len(test_feats), 2))

def objective(trial):

    best_score = np.inf
    best_model = None

    params={
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10.0),
        # 'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10.0),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        # 'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        # 'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        # 'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        # 'colsample_bynode': trial.suggest_float('colsample_bynode', 0.4, 1.0),
        # 'eta': trial.suggest_loguniform('eta', 1e-2, 0.5),
        # 'max_depth': trial.suggest_int('max_depth', 1, 10),
        # 'random_state': 42,
        # 'n_jobs': 4,
        # 'eval_metric': 'rmse',
        'verbosity': 0
    }

    oof = np.zeros(len(train_feats))
    kf = model_selection.KFold(n_splits=5, random_state=42, shuffle=True)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):
        X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
        X_valid, y_valid = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]

        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

        preds = model.predict(X_valid)
        oof[valid_idx] = preds

        score = metrics.mean_squared_error(y_valid, preds, squared=False)
        if score < best_score:
            best_score = score
            best_model = model

    joblib.dump(best_model, MODEL_DIR / f'model_{trial.number}.pkl')

    rmse = metrics.mean_squared_error(train_feats[target_col], oof, squared=False)
    return rmse

In [273]:
sampler = TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=1)

[I 2024-01-04 21:41:09,545] A new study created in memory with name: no-name-7729e394-b020-40fd-88c2-c32822022d80
[W 2024-01-04 21:41:12,019] Trial 0 failed with parameters: {'reg_alpha': 0.0017670169402947942} because of the following error: XGBoostError('value -1 for Parameter verbosity exceed bound [0,3]\nverbosity: Flag to print out detailed breakdown of runtime.').
Traceback (most recent call last):
  File "/home/ljy/anaconda3/envs/kaggle/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1346553/3304381916.py", line 42, in objective
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
  File "/home/ljy/anaconda3/envs/kaggle/lib/python3.10/site-packages/xgboost/core.py", line 730, in inner_f
  File "/home/ljy/anaconda3/envs/kaggle/lib/python3.10/site-packages/xgboost/sklearn.py", line 1053, in fit
    X=X,
  File "/home/ljy/anaconda3/envs/kaggle/lib/python3.10/contextlib

XGBoostError: value -1 for Parameter verbosity exceed bound [0,3]
verbosity: Flag to print out detailed breakdown of runtime.

In [ ]:
# Delete all model files except for the best one
for trial in study.trials:
    if trial.number != study.best_trial.number:
        os.remove(MODEL_DIR /  f'model_{trial.number}.pkl')

In [136]:
print("Best score: ", study.best_trial.values)

Best score:  [0.6108848360170163]


In [ ]:
best_model = joblib.load(MODEL_DIR / f'model_{study.best_trial.number}.pkl')

In [ ]:
# Calculate feature importances
feature_importances_values = best_model.feature_importances_
feature_importance_df = pd.DataFrame({'name': train_cols, 'importance': feature_importances_values}).sort_values('importance', ascending=False)

# save feature importance
feature_importance_df.to_csv(Feature_DIR / f'feature_importance_{VER}.csv', index=False)

In [ ]:
print(feature_importance_df.query('importance >= @fea_score_door'))

In [ ]:
# print('OOF metric LGBM = {:.5f}'.format(metrics.mean_squared_error(train_feats[target_col], 
#                                                                    OOF_PREDS[:, 0], 
#                                                                    squared=False)))